In [5]:
# =====================================
# Colab-ready CNN Training on RSS Maps
# =====================================

# ---------- Step 1: Setup ----------
!pip install -q matplotlib pandas numpy torch torchvision scikit-learn

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from google.colab import files

# ---------- Step 2: Upload RSS Files ----------
print("Please upload your RSS files (e.g., rss_munich_*.csv or *.csv.gz):")
uploaded = files.upload()
rss_dir = "/content"  # Default upload path in Colab

# ---------- Step 3: Load RSS Maps ----------
def load_rss_maps_from_dir(rss_dir):
    rss_maps = []
    tx_coords = []
    for file in os.listdir(rss_dir):
        if file.endswith('.csv') or file.endswith('.csv.gz'):
            filepath = os.path.join(rss_dir, file)
            try:
                rss = pd.read_csv(filepath, header=None).values
                rss_maps.append(rss)

                coord = file.replace('rss_munich_', '').replace('.csv.gz', '').replace('.csv', '')
                x, y, z = map(float, coord.split(','))
                tx_coords.append([x, y])
            except Exception as e:
                print(f"Skipping file {file} due to error: {e}")
    return np.array(rss_maps), np.array(tx_coords)

# ---------- Step 4: Dataset ----------
class RSSMapDataset(Dataset):
    def __init__(self, rss_maps, tx_coords):
        self.tx_coords_max = np.max(tx_coords, axis=0)
        self.rss_maps = (rss_maps - np.min(rss_maps)) / (np.max(rss_maps) - np.min(rss_maps))  # Normalize RSS
        self.tx_coords = tx_coords / self.tx_coords_max  # Normalize positions

    def __len__(self):
        return len(self.rss_maps)

    def __getitem__(self, idx):
        x = torch.tensor(self.rss_maps[idx], dtype=torch.float32).unsqueeze(0)  # Shape: (1, H, W)
        y = torch.tensor(self.tx_coords[idx], dtype=torch.float32)  # Shape: (2,)
        return x, y

# ---------- Step 5: CNN Model ----------
class RSSCNN(nn.Module):
    def __init__(self):
        super(RSSCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(32, 2)  # Predict (x, y)
        )

    def forward(self, x):
        return self.model(x)

# ---------- Step 6: Training Loop ----------
def train_model(model, dataloader, epochs=20):
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        for x, y in dataloader:
            pred = model(x)
            loss = criterion(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}: Loss = {total_loss / len(dataloader):.4f}")

# ---------- Step 7: Load Data & Train ----------
rss_maps, tx_coords = load_rss_maps_from_dir(rss_dir)

X_train, X_test, y_train, y_test = train_test_split(
    rss_maps, tx_coords, test_size=0.2, random_state=42
)

train_dataset = RSSMapDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

model = RSSCNN()
train_model(model, train_loader, epochs=20)

# ---------- Step 8: Evaluate ----------
model.eval()
with torch.no_grad():
    sample = torch.tensor(X_test[0], dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # (1, 1, H, W)
    pred_coord = model(sample).squeeze().numpy()
    true_coord = y_test[0] / np.max(tx_coords, axis=0)

    print("\n--- Evaluation ---")
    print("Predicted Tx Location (normalized):", pred_coord)
    print("True Tx Location (normalized):     ", true_coord)


ERROR: Operation cancelled by user


KeyboardInterrupt: 